<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline


# Baseline

In [41]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [42]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [43]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## Load DB

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import numpy as np


UPSTAGE_API_KEY = api_key

# .npy 파일 로드 (타입==넘파이)
ewhaDB = np.load(data_path+f'embedding/full_ewha500.npy')

ewhaDB_embed = np.load(data_path+f'embedding/full_ewha500_embed.npy')
ewhaDB_embed = ewhaDB_embed.tolist()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


## DB Split (chunking)

## test set 가져와서 dataframe 만들기

In [112]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [113]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [114]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [172]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers','predict', 'top1', 'top2', 'top3', 'top1plus', 'top2plus', 'top3plus', 'ground' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # >>질문<<만 받아와서 embedding 하기 // >>선지까지<< 받아와서 embedding하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None, 'predict': None, 'gound': None }

    except :
        print(f'pass: {index}')
        continue 


In [173]:
nowtest

,index,embed_ques,question,prompts,answers,predict,top1,top2,top3,top1plus,top2plus,top3plus,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None,None,NaN,NaN,NaN,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None,None,NaN,NaN,NaN,NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None,None,NaN,NaN,NaN,NaN
3,3,"[-0.002178192138671875, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None,None,NaN,NaN,NaN,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None,None,NaN,NaN,NaN,NaN
5,5,"[0.0015668869018554688, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None,None,NaN,NaN,NaN,NaN
6,6,"[-0.0182342529296875, -0.0157318115234375, -0....",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None,None,NaN,NaN,NaN,NaN
7,7,"[0.0028839111328125, -0.018707275390625, -0.03...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None,None,NaN,NaN,NaN,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None,None,NaN,NaN,NaN,NaN
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None,None,NaN,NaN,NaN,NaN


## chunk마다 Evidence와 Reasoning 생성

계산 효율을 위해 그냥 모든 chunk를 미리 segment+(note) 해두기 (나중에 유사도 계산 후에 관련있는 애들 다시 중복으로 계산 할 바에 미리 해둔다는 뜻)

In [174]:
############# Segment+ first prompt ( Evidence and Reasoning ) ##########

### 논문에서 제공하는 영어 버전으로 하면, 답변에 영어가 섞이는 현상 -> 프롬프트를 한글로 바꿈
### 논문 그대로 프롬프트를 제공하면 전부 이해하지 못하는 것 같아서 좀 간소화시킴

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    이제부터 너는 대학교 학칙에 대한 질문에 답변을 해야 돼.

    너에게 학칙 일부와 질문을 제공할게.
    Segment Content: {context1}
    —
    Question: {question}
    —
    너가 할 일: 제공한 학칙의 일부분과 질문을 비교해서, 학칙 정보를 [증거] 그리고 [추론] 두 가지로 다시 정리해.
    다음 지시사항을 따라서 답변해: 
    [증거] :
        1. 질문과 관련된 학칙의 주요 문장이나 설명을 추출하라. 숫자, 관련 단어 등 중요한 세부 사항에 중점을 둔다.
        2. 질문과 직접적으로 관련된 문장을 정확하게 추출하라.
    [추론]:
        1. 질문을 분석하여 답변, 요약, 해석 또는 질문을 깊이 이해하는 데 도움이 되는 주석을 작성하라.
    다음과 같은 형식으로 답변해 :
        [ '증거': '추출한 내용' , '추론': '추론한 내용']
        
    '''

)
segplus1 = prompt_template | llm


In [175]:
nowtest.question[0]

'영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?\n'

## question - chunk 유사도 계산

In [ ]:
# 유사도 높은 chunk top3를 df에 넣어두기

import numpy as np


for idx, row in nowtest.iterrows() : # 질문 받아오기 
    #if idx == 10 : break ## 테스트용
    embed_ques= row.embed_ques
    prompt = row.prompts
    ques = row.question

    sorted_idx = (np.array(embed_ques) @ np.array(ewhaDB_embed).T).argsort()[::-1]

    print( f" ==================={idx+1}==================== ")
    for topidx in range(3) :
        top = f"top{topidx+1}"
        topplus = f"top{topidx+1}plus"

        nowtest.loc[idx, top] = ewhaDB[sorted_idx[topidx]]

        response = segplus1.invoke({"context1": ewhaDB[sorted_idx[topidx]], "question": prompt})
        nowtest.loc[idx, topplus] = response.content
        print(response.content) # 확인용
        print("***")
    


 ===================1==================== 


/tmp/ipykernel_4038762/4154707990.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[증거: 제48조의2(영어 및 정보인증) ① 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다.], ['추론: 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우, 각각 3학점으로 인정받을 수 있습니다.']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, topplus] = response.content


[증거: 제48조의2(영어 및 정보인증) ① 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다.], ['추론: 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우, 각각 3학점으로 인정받을 수 있습니다.']
***


/tmp/ipykernel_4038762/4154707990.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[증거': '제35조의2(학점취득의 특례) ① 교양과목과 전공기초과목 중 특정교과목은 이를 수강하지 아니하여도 학점취득특별시험에 의하여 학기당 최대취득학점 외에 학점을 취득할 수 있으며, 그 학점수는 12학점을 초과할 수 없다.', '추론': '영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우, 학점취득특별시험을 통해 학기당 최대취득학점 외에 12학점을 초과하지 않는 범위 내에서 학점을 취득할 수 있습니다.']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, topplus] = response.content


[증거': '제35조의2(학점취득의 특례) ① 교양과목과 전공기초과목 중 특정교과목은 이를 수강하지 아니하여도 학점취득특별시험에 의하여 학기당 최대취득학점 외에 학점을 취득할 수 있으며, 그 학점수는 12학점을 초과할 수 없다.', '추론': '영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우, 학점취득특별시험을 통해 학기당 최대취득학점 외에 12학점을 초과하지 않는 범위 내에서 학점을 취득할 수 있습니다.']
***


/tmp/ipykernel_4038762/4154707990.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[증거': '제45조(학기당 취득학점) ① 일반학기의 취득기준학점은 18학점으로 한다. 다만, 다음 각 호의 경우: 21학점까지 취득'

[추론': 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 학칙의 일부분에는 직접적으로 언급되지 않습니다. 그러나, 학칙에 따르면, 특별 사유가 있거나 교과과정 운영상 필요한 경우, 또는 특정 조건을 충족하는 경우 (예: 직전 학기 평균성적 3.75 이상, 학사편입학, 학·석사 연계과정, 의과대학 의학과, 약학대학 등)에는 최대 21학점까지 취득할 수 있습니다. 따라서, 영어 및 정보 등에 관한 일정한 기준의 능력이나 자격을 취득한 경우, 해당 사유에 해당하는 경우 최대 21학점까지 인정 받을 수 있을 수 있습니다. 그러나, 해당 학칙의 부분에서는 영어 및 정보 등에 관한 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 구체적인 언급이 없으므로, 정확한 답변을 제공하기 어렵습니다.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, topplus] = response.content


[증거': '제45조(학기당 취득학점) ① 일반학기의 취득기준학점은 18학점으로 한다. 다만, 다음 각 호의 경우: 21학점까지 취득'

[추론': 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 학칙의 일부분에는 직접적으로 언급되지 않습니다. 그러나, 학칙에 따르면, 특별 사유가 있거나 교과과정 운영상 필요한 경우, 또는 특정 조건을 충족하는 경우 (예: 직전 학기 평균성적 3.75 이상, 학사편입학, 학·석사 연계과정, 의과대학 의학과, 약학대학 등)에는 최대 21학점까지 취득할 수 있습니다. 따라서, 영어 및 정보 등에 관한 일정한 기준의 능력이나 자격을 취득한 경우, 해당 사유에 해당하는 경우 최대 21학점까지 인정 받을 수 있을 수 있습니다. 그러나, 해당 학칙의 부분에서는 영어 및 정보 등에 관한 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점에 대한 구체적인 언급이 없으므로, 정확한 답변을 제공하기 어렵습니다.
***
 ===================2==================== 
[증거]: "학사학위의 종류(제50조 관련) 대 학 학위의 종류 학과 또는 전공"
[추론]: 이 학칙은 해당 대학의 학사학위 종류와 학과 또는 전공에 대한 정보를 제공합니다. 질문은 각 대학에 따른 학위의 종류로 해당하지 않는 것을 찾는 것입니다. 주어진 학칙 정보를 바탕으로, 간호학사, 인공지능학사, 가정학사, 무용학사, 음악학사는 모두 해당 대학의 학과 또는 전공에 대한 학사학위 종류로 언급되었습니다. 반면, 공학사는 주어진 학칙 정보에서 언급되지 않았으므로, 각 대학에 따른 학위의 종류로 해당하지 않는 것은 (C) 공학사입니다.
***
[증거]: "제35조의5(국내외 연수 및 현장실습 학점의 인정) ① 각 대학장은 교육과정위원회의 심의를 거쳐 총장의 승인을 얻어 국내외 연수를 시행하고, 학기당 취득학점 범위 안에서 학점을 취득하게 할 수 있다. (개정 2017.5.15.)② 국내외의 기관(기업 포함

In [177]:
nowtest.top3[1]

'교육 전공 \n 초등특수교육 전공 \n 중등특수교육 전공 \n 영어교육과 42\n 사회과교육과 84\n 역사교육 전공 \n 사회교육 전공 \n 지리교육 전공 \n 국어교육과 30\n 과학교육과 104\n 물리교육 전공 \n 화학교육 전공 생물교육 전공 \n 지구과학교육 전공 \n 수학교육과 32\n  계 467\n 경영대학 경영학부 경영학 전공 132\n 국제사무학과 25\n  계 157\n 건강과학대학 간호학부 체육과학부 간호학 전공 78\n 글로벌건강간호학 전공 \n 체육과학 전공 60\n 식품영양학과 50\n 보건관리학과 31\n  계 219\n 약학대학  약학과 120\n  계 120\n 스크랜튼대학 스크랜튼학부 국제학부 자유 전공 40\n 국제학 전공 80\n  계 120\n 총계 3,109[별표 2] (개정 2014.11.21., 2015.9.18., 2016.2.16., 2016.3.31., 2017.2.8.)\n학사학위의 종류(제50조 관련)대 학 학위의 종류 학과 또는 전공\n 인 문 과 학 대 학 문 학 사 국 어 국 문 학 중 어 중 문 학 불 어 불 문 학 독 어 독 문 학 사 학 철 학 기 독 교 학 영 어 문 학\n 사 회 과 학 대 학 정 치 학 사 행 정 학 사 경 제 학 사 문 학 사 커 사 영 정 치 외 교 학 행 정 학 경 제 학 문 헌 정 보 학 사 회 학 사 회 복 지 학 심 리 학 소 비 자 학 커뮤니케이션‧미디어학\n 자 연 과 학 대 학 뮤 니 케 이 션 학 이 학 사 수 학 통 계 학 물 리 학 화 학 · 나 노 과 학 생 명 과 학\n 엘 텍 공 과 대 학 공 학 사 휴 먼 기 계 바 이 오 공 학 컴 퓨 터 공 학 사 이 버 보 안 전 자 전 기 공 학 화 학 신 소 재 공 학 식 품 공 학 기후·에너지시스템공학 환 경 공 학 건 축 도 시 시 스 템 공 학\n 음 악 대 학 건 축 학 사 음 악 학 사 건 축 학 건 반 악 기 관 현 악 성 악 작 곡\n  무 사 한 국 음 악 무 용\n 조 형 예 술 대 학

In [178]:
nowtest

,index,embed_ques,question,prompts,answers,predict,top1,top2,top3,top1plus,top2plus,top3plus,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,을 이수한 것으로 인정한다. (개정 1998.6.23)\n④ 삭제 (1998.6.2...,하지 아니한 자\n4. 성적이 불량한 자 (개정 2002.11.8)\n5. 재학연한...,실시할 수 있다.\n(개정 1981.5.19)의 경우에는 그러하지 아니하다. (개정...,[증거: 제48조의2(영어 및 정보인증) ① 영어 및 정보 등에 관하여 일정한 기준...,[증거': '제35조의2(학점취득의 특례) ① 교양과목과 전공기초과목 중 특정교과목...,[증거': '제45조(학기당 취득학점) ① 일반학기의 취득기준학점은 18학점으로 한...,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,화학신소재공학부 화학신소재공학 전공 60\n 음악대학 음악학부 계 건반악기 전공 관...,하지 아니한 자\n4. 성적이 불량한 자 (개정 2002.11.8)\n5. 재학연한...,교육 전공 \n 초등특수교육 전공 \n 중등특수교육 전공 \n 영어교육과 42\n ...,"[증거]: ""학사학위의 종류(제50조 관련) 대 학 학위의 종류 학과 또는 전공""\...","[증거]: ""제35조의5(국내외 연수 및 현장실습 학점의 인정) ① 각 대학장은 교...","[증거]: ""경영학과 국제사무학과, 간호학과 글로벌건강간호학과, 체육학과, 식품영양...",NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,교의 교무회의에 관한 사항은 따로 정한다.제16장 자체평가 (신설 2009.9.22...,는 시점에서 이를 폐지한다.부칙(2005. 2. 11 개정)\n이 학칙은 2005년...,"교육학과, 유아\n교육과, 초등교육과 및 수학교육과의 재적생으로 본다.부칙(학무25...","[증거]: ""부칙(문고대 1041.3-171, 1967. 1. 24 개정) 본 학칙...","[증거]: ""② (전공 명칭 변경에 따른 경과조치) 구 학칙 시행당시 조형예술대학 ...","[증거]: ""이 학칙은 1993년 7월 10일부터 시행한다.""\n""이 학칙은 199...",NaN
3,3,"[-0.002178192138671875, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,과정을 이\n수한 경우 입학 후 대학의 학점으로 인정할 수 있다. 학점인정대상자 및...,"의 성적등급이 C+이하인 경우, 해당 교과목을 재수강할 수 있으며,\n재수강한 교과...","제1항의 수료사정에 있어서는 총 평균성적이 1.70 이상이어야 하며, 기타 사항은 ...","[증거': '성적은 매 학기 수업이 개시되기 전에 총장이 정한다.', '추론': '...","[증거': '학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?', '...","[증거]: ""제50조(졸업) ① 이 학칙 소정의 과정을 이수하고 일정한 시험에 합격...",NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정제1장 ...,다)\n을 납부하여야 한다. (개정 2011.2.24.)\n제60조의2(등록금심의위...,"의하여 설치․운영하는 계약학과는 별표 3과 같다. (개정 2014.5.15., 20...",[증거': '본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이...,"[증거': '이화여자대학교는 기독교 정신을 바탕으로 교육, 연구, 봉사를 통하여 국...","[증거]: ""제1조(목적) 이 대학은 기독교 정신을 바탕으로 교육하여 국가와 인류사...",NaN
5,5,"[0.0015668869018554688, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정제1장 ...,다)\n을 납부하여야 한다. (개정 2011.2.24.)\n제60조의2(등록금심의위...,"의하여 설치․운영하는 계약학과는 별표 3과 같다. (개정 2014.5.15., 20...","[증거]: ""제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개...","[증거]: ""본교는 서울특별시 서대문구 이화여대길 52에 위치한다.""\n[추론]: ...","[증거: 제1조(소재지) 본교는 서울특별시 서대문구 이화여대길 52에 둔다.], [...",NaN
6,6,"[-0.0182342529296875, -0.0157318115234375, -0....",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,"수하는 학생, 「고등교육법 시행령」 제29조제2항의제6호 및 제7호에 해당\n하는 ...","의하여 설치․운영하는 계약학과는 별표 3과 같다. (개정 2014.5.15., 20...",2.23)\n③ 삭제 (1997.12.23)\n④ 삭제 (1997.12.23)\n⑤...,[증거': '학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 ...,"[증거]: ""제12조의2(교원의 교수시간) 교원의 교수시간은 매 학년도 30주를 기...","[증거]: 학칙 제12조에는 임시휴업을 결정할 수 있는 근거로 국가적 비상사태, 자...",NaN
7,7,"[0.0028839111328125, -0.018707275390625, -0.03...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,당하게 할 수 있다.② 입학사정관의 조직과 운영에 관한 사항은 총장이 따로 정한다....,"수하는 학생, 「고등교육법 시행령」 제29조제2항의제6호 및 제7호에 해당\n하는 ...",이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정제1장 ...,"[증거]: ""① 각 대학의 수업연한은 4년으로 한다. 다만, 엘텍공과대학의 건축학전...","[증거: '제23조(수업연한) ④ 건축학전공의 경우 4년, 의예과의 경우 3학기',...","[증거]:\n\n* 학칙 제23조: ""건축학과는 5년, 약학대학은 6년의 수업연한을...",NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 

In [179]:
############# Segment+ 2nd Prompt ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    주어진 [Context]에서 관련된 정보를 찾아서 multiple choice [Question]의 정답을 출력해
    ---
    
    [Question] {question}
    [Context] {context1} {context2} {context3}
    [Answer]
        
    '''

)
segplus2 = prompt_template | llm


responses = []
for idx, row in nowtest.iterrows() : 
        
    response = segplus2.invoke({"question": row.prompts, "context1": row.top1plus, "context2": row.top2plus, "context3": row.top3plus}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

responses

['D',
 '(C) 공학사',
 '(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다',
 '(B) 성적 입력 마감 후',
 '(C) 지역사회의 봉사',
 '(D) 강남구입니다.',
 '(C) 학생총회 요청',
 'B',
 '(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능',
 '(B) 전공별 종합시험 합격',
 'E',
 'C',
 '(D) 타교에 입학한 경우',
 'D',
 '(A)',
 '(C) 15시간 이상',
 '(E) 특정 종교 교리 확립',
 '정답은 (C) 전공 결정은 일정 학점 이수 후 이루어진다입니다.',
 '질문에 대한 정답은 (D) 엘텍공과대학 건축학전공입니다. 주어진 학칙에 따르면, 엘텍공과대학의 건축학전공은 수업 연한이 4년이 아닌 5년으로 규정되어 있습니다.',
 '(C) 21학점',
 '(C) 학기 중간에 전과를 신청한 경우',
 '(C) 동기방학',
 '(B)',
 '답변:\nC',
 '(B) 시험 부정행위']

## check accuracy

In [182]:
######### 정답 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong2 = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong2.append(idx) 
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong2.append(idx)
        

print()
print(f"acc: {(cnt/len(answers))*100}%")
print()
print("wrong:",wrong2)

----------
D
idx: 0 | generated answer: D, answer: (D)
----------
(C) 공학사
idx: 1 | generated answer: C, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
(B) 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
(C) 지역사회의 봉사
idx: 4 | generated answer: C, answer: (C)
----------
(D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
B
idx: 7 | generated answer: B, answer:  (D)
----------
(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능
idx: 8 | generated answer: E, answer:  (C)
----------
(B) 전공별 종합시험 합격
idx: 9 | generated answer: B, answer:  (B)
----------
E
idx: 10 | generated answer: E, answer:  (B)
----------
C
idx: 11 | generated answer: C, answer:  (C)
----------
(D) 타교에 입학한 경우
idx: 12 | generated answer: D, answer:  (B)
----------
D
idx: 13 | generated answer: D, answer:  (E)
----------
(A)
idx: 14 | generated answer: A, answer:  (A)
----------
(C)

## groundedness check

### 질문할 때마다 답이 달라져서 안 할 거임 

In [ ]:
import os
from langchain_upstage import UpstageGroundednessCheck # langchain_upstage==0.1.3
 
groundedness_check = UpstageGroundednessCheck()
 
request_input = {
    "context": f"Mauna Kea is an inactive volcano on the island of Hawai'i. Its peak is 4,207.3 m above sea level, making it the highest point in Hawaii and second-highest peak of an island on Earth.",
    "answer": "Mauna Kea is 5,207.3 meters tall.",
}
response = groundedness_check.invoke(request_input)
print(response)